In [4]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [5]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [6]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SEQ_TRANSACTION_ID', 'des_col': 'seq_transaction_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNT_NO', 'des_col': 'account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ACCOUNT_NO', 'des_col': 'new_account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY', 'des_col': 'category', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROUTE', 'des_col': 'route', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ROUTE_ID', 'des_col': 'route_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BINDER_PAGE_NO', 'des_col': 'binder_page_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBGROUP', 'des_col': 'subgroup', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOAD_TYPE', 'des_col': 'load_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCENARIO', 'des_col': 'scenario', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FROM_OFFICE_ID', 'des_col': 'from_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FROM_READER_ID', 'des_col': 'from_reader_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FROM_DTR_ID', 'des_col': 'from_dtr_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FROM_POLE_ID', 'des_col': 'from_pole_id', 'src_dtype': 'String', 'des_dtype': 'String'},  
    {'src_col': 'TO_OFFICE_ID', 'des_col': 'to_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'T0_READER_ID', 'des_col': 't0_reader_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'T0_DTR_ID', 'des_col': 't0_dtr_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'T0_POLE_ID', 'des_col': 't0_pole_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'}
     
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.CONSUMER_ROUTE_HISTORY', 'public', 'consumer_route_history', cols_map)
print("Total Migrated Records (consumer_route_history): " + str(migrated_row_count))


Total Migrated Records (consumer_route_history): Total Src Records: 219047Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1226, in _execute_context
    cursor, statement, parameters, context
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\sqlalchemy\dialects\postgresql\psycopg2.py", line 854, in do_executemany
    cursor.executemany(statement, parameters)
psycopg2.errors.UniqueViolation: duplicate key value violates unique constraint "consumer_route_history_pkey"
DETAIL:  Key (id)=(1) already exists.


The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Lenovo\Desktop\migration\my_functions.py", line 54, in migrate_data
    dest_result_set.to_sql(desc_table, con = dest_engine, schema=desc_schema, index=False, if_exists = 'append', index_label=desc_index)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\generic.py", line 2663, i